In [40]:
# univariate multi-step lstm
import pandas as pd
import numpy as np
import scipy.stats as st
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import (LSTM, Dense, Flatten)
from keras.utils.np_utils import to_categorical

In [3]:
df = read_csv("predicted_temperature_and_test_values.csv")

In [4]:
df.head(5)

,pred_temperature,temperature,humidity
0,23.294163,24.000000,17.241379
1,23.227598,24.000000,17.016949
2,23.374884,24.000000,16.966667
3,23.404564,24.000000,16.357143
4,23.345507,24.389831,15.576271


In [33]:
def calculate_confidence(data):
    diff_actual_pred = data.temperature - data.pred_temperature
    confidence_interval= st.t.interval(0.99, diff_actual_pred.shape[0]-1, loc=diff_actual_pred.mean(), scale=st.sem(diff_actual_pred.values.tolist()))
    return confidence_interval

In [34]:
interval = calculate_confidence(df)
print(interval, (interval[1] - interval[0])/2)

(0.19071296071307547, 0.8509075109596698) 0.3300972751232971


In [37]:
def to_labeled(data):
    diff_actual_pred = data.temperature - data.pred_temperature
    interval = calculate_confidence(data)
    data['label'] = ((diff_actual_pred>interval[0]) & (diff_actual_pred<interval[1]))

In [45]:
# split a univariate dataset into train/test sets
def split_dataset(data):
    split_index = int((len(data)*0.8))
    train, test = data[:split_index], data[split_index:]
    return train, test

In [76]:
# train the model
def build_model(train):
    train_x = train[["temperature", "humidity"]].values.reshape((-1, 1, 2))
    train_y = to_categorical(train.label.values, num_classes=2)
    # define parameters
    verbose, epochs, batch_size = 1, 10, 5
    n_features, n_outputs = train_x.shape[2], train_y.shape[1]
    n_timesteps = 1
    print(n_outputs, n_timesteps, n_features)
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [79]:
to_labeled(df)
train, test = split_dataset(df)
build_model(train).predict(test[["temperature", "humidity"]].values.reshape((-1, 1, 2)))

2 1 2
Epoch 1/10
8/8 [==============================] - 0s 2ms/step - loss: 0.2399
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 0.2083
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1940
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1729
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1631
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1596
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1618
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1642
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1582
Epoch 10/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1597


array([[0.76000893, 0.24321869],
       [0.7371352 , 0.26281118],
       [0.74622357, 0.24919342],
       [0.74538994, 0.25042772],
       [0.73382497, 0.267528  ],
       [0.7411209 , 0.25679874],
       [0.738291  , 0.26106843],
       [0.74667454, 0.24852704],
       [0.75895786, 0.2307301 ],
       [0.74667454, 0.24852704]], dtype=float32)